In [1]:
#Load Libraries
import numpy as np
import pandas as pd

In [2]:
#Load Dataset
data_Unclean= pd.read_csv('./2021_ontario_public_library_statistics_open_data.csv', encoding='latin1')
data_Unclean.head() 

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 320,Unnamed: 321,Unnamed: 322,Unnamed: 323,Unnamed: 324,Unnamed: 325,Unnamed: 326,Unnamed: 327,Unnamed: 328,Unnamed: 329
0,Library Full Name,Library Number,Survey Year From,A1.3 Ontario Library Service (OLS) Region (Eng...,A1.4 Type of Library Service (English),A1.5 Mailing Address,A1.9 Street Address,A1.10 City/Town,A1.11 Province,A1.12 Postal Code,...,H1.9.1.T Yes,H1.9.2.51 Chamber of Commerce,H1.9.2.52 Business Improvement Area (BIA),H1.9.2.95 Other Business and Economic Sector ...,H1.9.3 Please provide highlights or examples ...,P1.1 Resident Population Served,P1.2 Contracting Population Served,P2.1 Contracting Households Served,P2.2 Resident Households Served,NaN
1,Addington Highlands Twp,L0005,2021,Southern Ontario Library Service,Public or Union Library,3641 Flinton Rd. Box 72,3641 Flinton Rd.,Flinton,Ontario,K0H1P0,...,0,0,0,0,0,"1,712",0,0,"2,762",0.0
2,Adjala-TosorontioTwp,L0003,2021,Southern Ontario Library Service,Contracting Municipality,7855 Sideroad 30 R.R. #1,7855 Sideroad 30 R.R. #1,Alliston,Ontario,L9R1V1,...,0,0,0,0,0,"10,975",0,0,"4,103",0.0
3,Admaston/Bromley Twp,L0002,2021,Southern Ontario Library Service,Public or Union Library,Hwy 60 P O Box 130,5346 Highway 60,Douglas,Ontario,K0J1S0,...,0,0,0,0,0,"2,935",0,0,"1,408",0.0
4,Ajax,L0032,2021,Southern Ontario Library Service,Public or Union Library,55 Harwood Avenue South,55 Harwood Avenue South,Ajax,Ontario,L1S2H8,...,1,0,0,1,0,"127,400",0,0,"38,400",0.0


In [3]:
#Promote Headers
data_Headers = data_Unclean.rename(columns=data_Unclean.iloc[0]).drop(data_Unclean.index[0])
data_Headers.head()

#Remove Commas
data_Headers = data_Headers.replace(',',"",regex=True)

#Convert Data Types
c = data_Headers.select_dtypes(object).columns
data_Headers[c] = data_Headers[c].apply(pd.to_numeric,errors='coerce')

#Keep Only Funding and Population
data = data_Headers[['B2.1.4  Total Provincial Operating Funding','P1.1  Resident Population Served']]

#Drop na
data = data.dropna()

In [4]:
#Show Key Statistics
data.describe()

,B2.1.4 Total Provincial Operating Funding,P1.1 Resident Population Served
count,3.630000e+02,3.630000e+02
mean,6.212161e+04,3.868670e+04
std,3.124478e+05,1.789324e+05
min,2.480000e+02,0.000000e+00
25%,4.995500e+03,8.520000e+02
50%,1.430200e+04,4.107000e+03
75%,3.247050e+04,1.708500e+04
max,5.573927e+06,2.956024e+06


In [5]:
#Create Standard Model

#Define x and y variable
x = data.drop('B2.1.4  Total Provincial Operating Funding',axis=1).to_numpy()
y = data['B2.1.4  Total Provincial Operating Funding'].to_numpy()

#Create Train and Test Datasets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

#Fit model
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics

dec_tree = ExtraTreesRegressor(random_state=100)
dec_tree.fit(x_train,y_train)
predict = dec_tree.predict(x_train)

#Evaluate model
pred = dec_tree.predict(x_test)

print('R2: {:.2f}'.format(metrics.r2_score(y_test, pred))) 
adjusted_r_squared2 = 1-(1-metrics.r2_score(y_test, pred))*(len(y)-1)/(len(y)-x.shape[1]-1)
print('Adj_R2: {:0.2f}'.format(adjusted_r_squared2))
print('Mean Absolute Error: {:0.2f}'.format(metrics.mean_absolute_error(y_test, pred)))  
print('Mean Squared Error: {:0.2f}'.format(metrics.mean_squared_error(y_test, pred)))  
print('Root Mean Squared Error: {:0.2f}'.format(np.sqrt(metrics.mean_squared_error(y_test, pred)))) 

R2: 0.87
Adj_R2: 0.87
Mean Absolute Error: 17281.68
Mean Squared Error: 1221911142.32
Root Mean Squared Error: 34955.85


In [7]:
#Predict

#load data
predict_data= pd.read_csv('./predictdata.csv')
x = predict_data.to_numpy()

#Predict
new_predict = dec_tree.predict(x)
print(new_predict)


[ 271808.    216919.7   689990.72  854951.48  563128.8  1155598.22
 1799687.9  1883559.88 2177111.81 2554535.72 2931959.63 3183575.57
 3602935.47 3896487.4  4315847.3  4483591.26 4777143.19 5028759.13
 5448119.03 5573927.  ]


In [9]:
np.savetxt("testpredictresults.csv", new_predict, delimiter=",")